In [8]:
# Practice by modeling diffusion of Na+ vacancy in NaCl

# Import modules

import numpy as np
import matplotlib.pyplot as plt
from ase import Atoms
from ase.visualize import view
from kmc import *


In [ ]:

# Define simulation parameters


# num_steps = 10000
lattice_size = 10
temp = 1000
boltz = 8.617333262e-5
# values for activation energy and attempt frequency come from: https://pubs.aip.org/aip/jap/article/33/1/473/288425/Energy-and-Entropy-of-Formation-and-Motion-of
activation_energy = 0.8 
attempt_frequency = 10e14

# Calculate rate constant for cation movement (assume the rate constant is the same for all possible jumps)
rate_constant = calc_rate_constant(attempt_frequency, activation_energy, temp)

# Define the offsets for cation neighbors in rock salt structure
neighbor_offsets = [
    (-1, -1, 0), (1, -1, 0), (1, 1,0), (-1, 1, 0),
    (0, 1, 1), (0, 1, -1), (0, -1, 1), (0, -1, -1),
    (1, 0, 1), (-1, 0, 1), (-1, 0, -1), (1, 0, -1)
]
# Keep track of vacancy position and time elapsed
k = np.linspace(10000, 100000, 10, dtype=int)

# Set random seed
np.random.seed(42)

time_per_num_steps = []
diffusion_constants = []

# KMC procedure
for num_steps in k:
    # Initialize lattice then set vacancy at the center of this lattice
    lattice, vacancy_position = set_vacancy_center(initialize_lattice(lattice_size), lattice_size)
    
    # Keep track of vacancy position and time elapsed
    vacancy_positions = [tuple(vacancy_position)]
    time = 0
    time_steps = [time]

    for step in range(num_steps):
        print(f"num_steps: {num_steps:,} - step: {step:,}")
        while True:
            possible_moves = []
            for offset in neighbor_offsets: # Calculate coordinates for neighboring cation sites
                neighbor = [
                    (vacancy_position[i] + offset[i]) % lattice_size for i in range(3)
                ]
                # print(f"Offset: {offset}")
                possible_moves.append(neighbor)

            probability = [rate_constant for move in possible_moves]
            probability /= np.sum(probability) # Calculate the probability of jumping to each site (all jumps are equally probable in this scenario)

            chosen_move_idx = np.random.choice(len(possible_moves))
            # print(chosen_move_idx)
            # print(vacancy_position)
            possible_position = possible_moves[chosen_move_idx]

            xi = np.random.random()

            if xi < probability[chosen_move_idx]:
                break
        
        # print(possible_position)
        lattice[tuple(vacancy_position)] = 0
        lattice[tuple(possible_position)] = 1
        vacancy_position = possible_position
        vacancy_positions.append(tuple(vacancy_position))

        total_rate = np.sum([rate_constant for _ in possible_moves])
        time += -np.log(np.random.random()) / total_rate # Why not time += 1 / total_rate?
        time_steps.append(time)
        time_per_num_steps.append(time_steps)

    coeff = 1 / (6 * time)
    inside = np.mean((np.array(vacancy_positions[-1]) - np.array(vacancy_positions[0])) ** 2)
    diffusion_constant = coeff * inside
    diffusion_constants.append(diffusion_constant)

    
# # Convert the vacancy positions to a list of ASE atoms for visualtion

# list_atoms = []
# for position in vacancy_positions:
#     # Create a list of chemical symbols for the lattice sites

#     chemical_symbols = []
#     for i in range(lattice_size):
#         for j in range(lattice_size):
#             for k in range(lattice_size):
#                 if (i, j, k) == position:
#                     chemical_symbols.append('F') # Vacancy
#                 else:
#                     chemical_symbols.append('O')

#     atoms = Atoms(chemical_symbols, positions=[(i, j, k) for i in range(lattice_size) for j in range(lattice_size) for k in range(lattice_size)], pbc=True, cell=[lattice_size, lattice_size, lattice_size])
#     list_atoms.append(atoms)


# from ase.io import write
# write('vacancy_diffusion.extxyz', list_atoms)

In [ ]:
# Plot diffusion constant vs number of steps

plt.plot(k, diffusion_constants[:10])
plt.xlabel("# steps")
plt.ylabel("Diffusion constant (m^2/s)")
plt.title("Diffusion constant vs # of steps")
plt.savefig("DC_v_steps.png")


for i, num_steps in enumerate(k):
    print(f"The diffusion constant for K = {num_steps:,} is {diffusion_constants[i]:.2}")


# If lattice is not reinitialized every time
    #The diffusion constant for K = 10,000 is 1.6e+08
    #The diffusion constant for K = 20,000 is 2.1e+08
    #The diffusion constant for K = 30,000 is 9.5e+07
    #The diffusion constant for K = 40,000 is 2.8e+07
    #The diffusion constant for K = 50,000 is 9.2e+07
    #The diffusion constant for K = 60,000 is 4.3e+07
    #The diffusion constant for K = 70,000 is 4.6e+07
    #The diffusion constant for K = 80,000 is 2e+07
    #The diffusion constant for K = 90,000 is 5.3e+07
    #The diffusion constant for K = 100,000 is 1.8e+07

# If lattice is reinitialzied every time
    # The diffusion constant for K = 10,000 is 1.6e+08
    # The diffusion constant for K = 20,000 is 1.5e+08
    # The diffusion constant for K = 30,000 is 5.4e+07
    # The diffusion constant for K = 40,000 is 2.8e+07
    # The diffusion constant for K = 50,000 is 4.2e+07
    # The diffusion constant for K = 60,000 is 4.3e+07
    # The diffusion constant for K = 70,000 is 2.8e+07
    # The diffusion constant for K = 80,000 is 2e+07
    # The diffusion constant for K = 90,000 is 2.5e+07
    # The diffusion constant for K = 100,000 is 1.8e+07

In [ ]:
# Plot diffusion constant vs activation energy

diffusion_constants_2 = []
activation_energies = np.linspace(0, 10, 10)
num_steps = 100000

for ae in activation_energies:
    # Initialize lattice then set vacancy at the center of this lattice
    rate_constant = calc_rate_constant(attempt_frequency, ae, temp)
    lattice, vacancy_position = set_vacancy_center(initialize_lattice(lattice_size), lattice_size)

    # Keep track of vacancy position and time elapsed
    vacancy_positions = [tuple(vacancy_position)]
    time = 0
    time_steps = [time]

    for step in range(num_steps):
        print(f"Ea: {ae:.3} - step: {step:,}")
        while True:
            possible_moves = []
            for offset in neighbor_offsets: # Calculate coordinates for neighboring cation sites
                neighbor = [
                    (vacancy_position[i] + offset[i]) % lattice_size for i in range(3)
                ]
                # print(f"Offset: {offset}")
                possible_moves.append(neighbor)

            probability = [rate_constant for move in possible_moves]
            probability /= np.sum(probability) # Calculate the probability of jumping to each site (all jumps are equally probable in this scenario)

            chosen_move_idx = np.random.choice(len(possible_moves))
            # print(chosen_move_idx)
            # print(vacancy_position)
            possible_position = possible_moves[chosen_move_idx]

            xi = np.random.random()

            if xi < probability[chosen_move_idx]:
                break
        
        # print(possible_position)
        lattice[tuple(vacancy_position)] = 0
        lattice[tuple(possible_position)] = 1
        vacancy_position = possible_position
        vacancy_positions.append(tuple(vacancy_position))

        total_rate = np.sum([rate_constant for _ in possible_moves])
        time += -np.log(np.random.random()) / total_rate # Why not time += 1 / total_rate?
        time_steps.append(time)
        time_per_num_steps.append(time_steps)

    coeff = 1 / (6 * time)
    inside = np.mean((np.array(vacancy_positions[-1]) - np.array(vacancy_positions[0])) ** 2)
    diffusion_constant = coeff * inside
    diffusion_constants_2.append(diffusion_constant)


In [ ]:
# Plot diffusion constant vs activation energy

plt.plot(activation_energies, diffusion_constants_2)
plt.xlabel("Ea (eV)")
plt.ylabel(r"Diffusion constant ($m^2$/s)")
plt.title(r"Diffusion constant vs E_a")
plt.savefig("DC_v_Ea.png")

for i, ea in enumerate(activation_energies):
    print(f"The diffusion constant for Ea = {ea:.2} is {diffusion_constants_2[i]:.2}")